# Document Embeddings (Doc2Vec)

Existen multitud de maneras de abordar este problema. El que estudiaremos, y que se presenta a continuación, es Doc2Vec.

## Paragraph Vector: Distributed memory (PV-DM)

De manera similar a Word2Vec, en este caso los vectores representarán la información de un documento, en lugar de una palabra. En este caso, además de tener los vectores para cada palabra, se incluye un vector hace referencia al documento completo. De esta forma, cuando se calculan las palabras-vector, el vector para el documento es calculado también.

![pv-dm](../soluciones/img/PV-DM.png)

## Paragraph Vector: Distributed Bag of Words (PV-DBoW)

Análogo (o similar) al Skip-Gram en Word2Vec, el vector del documento es la entrada a la red, de manera que tratará de predecirse el contexto (todas las palabras que componen el documento).

![pv-dbow](../soluciones/img/PV-DBoW.png)

# Excercise: Document Clustering

In [1]:
import pandas as pd
import numpy as np
from time import time
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
from nltk import sent_tokenize
from nltk.corpus import stopwords
import string
from tqdm import tqdm
import re

In [3]:
string.punctuation += '“”«»¿¡‘’'
string.punctuation = string.punctuation.replace('-', '')
table = str.maketrans({key: None for key in string.punctuation})
print(string.punctuation)

!"#$%&'()*+,./:;<=>?@[\]^_`{|}~“”«»¿¡‘’


In [4]:
def preprocess(sent):
    return [token for token in re.sub("\n+", " ", str(sent).lower().translate(table)).split(" ")]

In [5]:
def doc_list_format(df, save=False, fname='train'):
    # List for appending documents. One line per document
    doc_list = []
    # Iterate over DataFrames and join preprocessed title and text in the same line
    for idx, (title, text) in enumerate(zip(df['title'], df['text'])):
        article_proc = " ".join(preprocess(title))
        try:
            for sent in sent_tokenize(text):
                article_proc += " " + " ".join(preprocess(sent))
        except:
            pass
        doc_list.append(article_proc)
        if save:
            with open('../../datasets/spanish_news_corpus_doc2vec_' + fname + '.txt', 'a') as f:
                f.write(article_proc + "\n")
        else:
            pass
    return doc_list

## Data Loading

In [6]:
%%time
df = pd.read_excel('../../datasets/spanish_news_corpus.xlsx')

CPU times: user 439 ms, sys: 7.33 s, total: 7.77 s
Wall time: 8.34 s


In [7]:
print(df.shape)
df.tail()

(39352, 6)


,date,title,text,keywords,media,url
39347,2019-10-23,Campofrío esquiva los aranceles de EEUU y la c...,La segunda mayor empresa de alimentación españ...,facturación|proteínas|esquiva|porcina|arancele...,expansion,https://www.expansion.com/empresas/distribucio...
39348,2019-10-23,"Capital Group aflora el 3,1% del capital de Ce...",La gestora de fondos estadounidense Capital Gr...,management|capital|través|euros|principal|comp...,expansion,https://www.expansion.com/mercados/2019/10/23/...
39349,2019-10-23,Se equivoca al rellenar la lotería y gana dos ...,Las personas que creen en el destino suelen pe...,vez|dólares|primer|ganador|dos|equivoca|número...,elconfidencial,https://www.elconfidencial.com/alma-corazon-vi...
39350,2019-10-23,"El Premio Nacional de Narrativa, por Milena Bu...",Leo con enorme alivio y regocijo las declaraci...,ser|vez|escritor|si|valor|solapa|prostitutas|b...,elperiodico,https://www.elperiodico.com/es/opinion/2019102...
39351,2019-10-24,¿Dónde ve oportunidades en los mercados? por V...,¿Dónde ve oportunidades en los mercados?\n\n¿S...,dónde|responderá|regístrese|ve|riesgos|puede|p...,expansion,https://www.expansion.com/encuentros/victor-de...


In [8]:
%%time
doc = doc_list_format(df, save=False, fname='train')

## Doc2Vec Training

```python
params = {'corpus_file': '../../datasets/spanish_news_corpus_doc.txt',
          'dm': 1,
          'vector_size': 300,
          'window':5,
          'min_count': 5,
          'sample': 1e-5,
          'hs': 0,
          'negative': 20,
          'dm_mean': 1,
          'epochs': 20,
          'workers': 6}
d2v = Doc2Vec(**params)
```

In [9]:
from gensim.models import Doc2Vec
d2v = Doc2Vec.load('../../trained_models/d2v_dbow_d300_mc5_w5.pkl')

In [11]:
doc_1 = d2v.infer_vector(['sánchez', 'repetición', 'elecciones', 'pacto', 'psoe', 'socialistas'])
doc_2 = d2v.infer_vector(['casado', 'rivera', 'no', 'investidura', 'abstención', 'pp', 'cs'])
doc_3 = d2v.infer_vector(['dana', 'temporal', 'gota', 'fría', 'alicante', 'lluvias', 'torrenciales'])
print(cosine_similarity(doc_1.reshape(1, -1), doc_2.reshape(1, -1))[0][0])
print(cosine_similarity(doc_1.reshape(1, -1), doc_3.reshape(1, -1))[0][0])

0.8111827
0.5287716


## Birch Clustering

In [12]:
from sklearn.cluster import Birch
from sklearn.metrics import silhouette_score
import itertools

In [13]:
np.random.seed(2019)
rnd_index = np.random.choice(range(len(doc)), size=10000, replace=False)

In [21]:
%%time
birch = Birch(branching_factor=50, threshold=0.5, n_clusters=13, compute_labels=True).fit(d2v.docvecs.vectors_docs[rnd_index])
labels = birch.labels_

CPU times: user 27.4 s, sys: 962 ms, total: 28.4 s
Wall time: 29 s


In [22]:
clusters = {n: df.loc[rnd_index[np.where(labels==n)[0].tolist()].tolist(), 'title'].values.tolist() for n in range(len(set(labels)))}

In [23]:
for k, v in clusters.items():
    print("#"*50)
    print("Cluster: {}".format(k))
    for i in np.random.choice(range(len(v)), size=10, replace=False).tolist():
        print("* {}".format(v[i]))

##################################################
Cluster: 0
* El poder de salvar vidas de la educación sexual
* Cuba: Colas kilométricas para poder echar gasolina
* Gana 4,1 millones de euros en la lotería tras cambiar un número que le recordaba a su exmarido
* Tommy Hilfiger lanza su segunda colección cápsula para hombre 'TommyXMercedes-Benz'
* Los años Anagrama
* Un viaje cinematográfico con el crítico Elvis Mitchell como guía
* Entre el amor y el espanto, por Daniel Fernández
* Antonio Banderas traslada el espíritu de Broadway a Málaga
* Camilo Sesto: Los mejores temas del cantante fallecido
* Karina Sainz Borgo: “No conozco otra cosa que no sea la violencia”
##################################################
Cluster: 1
* Fotos: Vandalizadas, censuradas y vilipendiadas: ¿qué tienen estas 10 obras de arte para despertar tanto rechazo?
* El exlíder ultraderechista austriaco abandona la política, acosado por el caso Ibiza
* Israel: Benjamin Netanyahu afirma que se anexionará el Valle